# Measure MAMBA and CNN

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import json
import os

In [3]:
from tqdm import tqdm

import numpy as np
import keras
from tensorflow import keras
from keras.datasets import cifar10
from __future__ import print_function
from keras.models import Sequential
from keras.models import save_model, load_model
from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

import keras.backend as K
K.clear_session()

# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
# from model import Mamba, ModelArgs  # Import your custom Mamba implementation
# Assuming the model classes are defined in `model.py`
import sys
from model import ImageMamba, ModelArgs

In [15]:
import matplotlib.pyplot as plt
import os
import json

def plot_metrics_from_json(cnn_json_path='New_CNN_Metrics.json', 
                          mamba_json_path='New_MAMBA_Metrics.json', 
                          save_dir='comparison_plots'):
    """Create comparative plots using metrics from JSON files"""
    # Load metrics from JSON files
    with open(cnn_json_path, 'r') as f:
        cnn_metrics = json.load(f)
    with open(mamba_json_path, 'r') as f:
        mamba_metrics = json.load(f)
    
    os.makedirs(save_dir, exist_ok=True)
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    epochs = cnn_metrics['epochs']
    
    # 1. Accuracy Comparison
    ax1.plot(epochs, cnn_metrics['train_accuracies'], 'b-', label='CNN Train')
    ax1.plot(epochs, cnn_metrics['test_accuracies'], 'b--', label='CNN Test')
    ax1.plot(epochs, mamba_metrics['train_accuracies'], 'r-', label='MAMBA Train')
    ax1.plot(epochs, mamba_metrics['test_accuracies'], 'r--', label='MAMBA Test')
    ax1.set_title('Accuracy Comparison')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Accuracy (%)')
    ax1.legend()
    ax1.grid(True)
    
    # 2. Loss Comparison
    ax2.plot(epochs, cnn_metrics['train_losses'], 'b-', label='CNN Train')
    ax2.plot(epochs, cnn_metrics['test_losses'], 'b--', label='CNN Test')
    ax2.plot(epochs, mamba_metrics['train_losses'], 'r-', label='MAMBA Train')
    ax2.plot(epochs, mamba_metrics['test_losses'], 'r--', label='MAMBA Test')
    ax2.set_title('Loss Comparison')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True)
    
    # 3. Confidence Comparison
    ax3.plot(epochs, cnn_metrics['train_confidences'], 'b-', label='CNN Train')
    ax3.plot(epochs, cnn_metrics['test_confidences'], 'b--', label='CNN Test')
    ax3.plot(epochs, mamba_metrics['train_confidences'], 'r-', label='MAMBA Train')
    ax3.plot(epochs, mamba_metrics['test_confidences'], 'r--', label='MAMBA Test')
    ax3.set_title('Average Confidence Comparison')
    ax3.set_xlabel('Epochs')
    ax3.set_ylabel('Confidence')
    ax3.legend()
    ax3.grid(True)
    
    # 4. Overfitting Analysis
    cnn_gap = [t - v for t, v in zip(cnn_metrics['train_accuracies'], cnn_metrics['test_accuracies'])]
    mamba_gap = [t - v for t, v in zip(mamba_metrics['train_accuracies'], mamba_metrics['test_accuracies'])]
    
    ax4.plot(epochs, cnn_gap, 'b-', label='CNN')
    ax4.plot(epochs, mamba_gap, 'r-', label='MAMBA')
    ax4.set_title('Overfitting Analysis (Train-Test Accuracy Gap)')
    ax4.set_xlabel('Epochs')
    ax4.set_ylabel('Accuracy Gap (%)')
    ax4.legend()
    ax4.grid(True)
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, 'model_comparisons.png'))
    print(f"Saved comparison plots to {os.path.join(save_dir, 'model_comparisons.png')}")
    plt.close()

In [5]:
def evaluate_checkpoint(model, loader, device):
    """Evaluate a model checkpoint comprehensively"""
    model.eval()
    total = 0
    correct = 0
    running_loss = 0
    confidences = []
    
    criterion = nn.CrossEntropyLoss()
    
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            logits, probabilities = model(inputs)
            loss = criterion(logits, labels.squeeze())
            
            _, predicted = torch.max(logits, 1)
            confidence, _ = torch.max(probabilities, 1)
            
            total += labels.size(0)
            correct += (predicted == labels.squeeze()).sum().item()
            running_loss += loss.item()
            confidences.extend(confidence.cpu().numpy())
    
    accuracy = (correct / total) * 100
    avg_loss = running_loss / len(loader)
    avg_confidence = np.mean(confidences)
    
    return {
        'accuracy': accuracy,
        'loss': avg_loss,
        'avg_confidence': avg_confidence,
        'confidences': confidences
    }

def reconstruct_metrics_from_checkpoints(model_class, checkpoint_base_path, 
                                       train_loader, test_loader, 
                                       json_save_path, is_mamba=False, device='cuda'):
    """Reconstruct metrics from saved checkpoints"""
    model = model_class().to(device)
    metrics = {
        'train_accuracies': [],
        'test_accuracies': [],
        'train_losses': [],
        'test_losses': [],
        'train_confidences': [],
        'test_confidences': [],
        'epochs': [],
        'train_confidence_distributions': [],
        'test_confidence_distributions': []
    }
    
    # Evaluate checkpoints at every 100 epochs
    for epoch in range(100, 1501, 100):
        # Different naming conventions for MAMBA and CNN
        if is_mamba:
            checkpoint_path = os.path.join(checkpoint_base_path, f'model_epoch_{epoch}.pt')
        else:
            checkpoint_path = os.path.join(checkpoint_base_path, f'cnn_model_epoch_{epoch}.pt')
            
        if not os.path.exists(checkpoint_path):
            print(f"Skipping epoch {epoch} - checkpoint not found at {checkpoint_path}")
            continue
            
        print(f"Processing epoch {epoch}")
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        
        # Evaluate on train and test sets
        train_metrics = evaluate_checkpoint(model, train_loader, device)
        test_metrics = evaluate_checkpoint(model, test_loader, device)
        
        # Store metrics
        metrics['epochs'].append(epoch)
        metrics['train_accuracies'].append(train_metrics['accuracy'])
        metrics['test_accuracies'].append(test_metrics['accuracy'])
        metrics['train_losses'].append(train_metrics['loss'])
        metrics['test_losses'].append(test_metrics['loss'])
        metrics['train_confidences'].append(train_metrics['avg_confidence'])
        metrics['test_confidences'].append(test_metrics['avg_confidence'])
        metrics['train_confidence_distributions'].append(train_metrics['confidences'])
        metrics['test_confidence_distributions'].append(test_metrics['confidences'])
    
    # Save metrics to JSON
    with open(json_save_path, 'w') as f:
        json_metrics = {
            'train_accuracies': [float(x) for x in metrics['train_accuracies']],
            'test_accuracies': [float(x) for x in metrics['test_accuracies']],
            'train_losses': [float(x) for x in metrics['train_losses']],
            'test_losses': [float(x) for x in metrics['test_losses']],
            'train_confidences': [float(x) for x in metrics['train_confidences']],
            'test_confidences': [float(x) for x in metrics['test_confidences']],
            'epochs': metrics['epochs'],
            'train_confidence_distributions_stats': [
                {
                    'mean': float(np.mean(dist)),
                    'std': float(np.std(dist)),
                    'min': float(np.min(dist)),
                    'max': float(np.max(dist)),
                    'median': float(np.median(dist)),
                    'q1': float(np.percentile(dist, 25)),
                    'q3': float(np.percentile(dist, 75))
                } for dist in metrics['train_confidence_distributions']
            ],
            'test_confidence_distributions_stats': [
                {
                    'mean': float(np.mean(dist)),
                    'std': float(np.std(dist)),
                    'min': float(np.min(dist)),
                    'max': float(np.max(dist)),
                    'median': float(np.median(dist)),
                    'q1': float(np.percentile(dist, 25)),
                    'q3': float(np.percentile(dist, 75))
                } for dist in metrics['test_confidence_distributions']
            ]
        }
        json.dump(json_metrics, f, indent=4)
    
    return metrics

In [6]:
# Load CIFAR-10 data
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

# Reshape and preprocess the CIFAR-10 dataset for PyTorch models
X_train = X_train.transpose(0, 3, 1, 2)  # Shape: (batch_size, channels, height, width)
X_test = X_test.transpose(0, 3, 1, 2)

# Convert data to float and normalize pixel values in the range [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Convert the train/test data into PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.long)
Y_test_tensor = torch.tensor(Y_test, dtype=torch.long)

# Your existing train loader code
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Add test loader
test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)  # Note: shuffle=False for test data

In [7]:
# Set the device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
def evaluate_checkpoint(model, loader, device):
    """Evaluate a model checkpoint comprehensively"""
    model.eval()
    total = 0
    correct = 0
    running_loss = 0
    confidences = []
    
    criterion = nn.CrossEntropyLoss()
    
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            logits, probabilities = model(inputs)
            loss = criterion(logits, labels.squeeze())
            
            _, predicted = torch.max(logits, 1)
            confidence, _ = torch.max(probabilities, 1)
            
            total += labels.size(0)
            correct += (predicted == labels.squeeze()).sum().item()
            running_loss += loss.item()
            confidences.extend(confidence.cpu().numpy())
    
    accuracy = (correct / total) * 100
    avg_loss = running_loss / len(loader)
    avg_confidence = np.mean(confidences)
    
    return {
        'accuracy': accuracy,
        'loss': avg_loss,
        'avg_confidence': avg_confidence,
        'confidences': confidences
    }

def reconstruct_metrics_from_checkpoints(model, checkpoint_base_path, 
                                       train_loader, test_loader, 
                                       json_save_path, is_mamba=False, device='cuda'):
    """Reconstruct metrics from saved checkpoints"""
    metrics = {
        'train_accuracies': [],
        'test_accuracies': [],
        'train_losses': [],
        'test_losses': [],
        'train_confidences': [],
        'test_confidences': [],
        'epochs': [],
        'train_confidence_distributions': [],
        'test_confidence_distributions': []
    }
    
    # Evaluate checkpoints at every 100 epochs
    for epoch in range(100, 1501, 100):
        # Different naming conventions for MAMBA and CNN
        if is_mamba:
            checkpoint_path = os.path.join(checkpoint_base_path, f'model_epoch_{epoch}.pt')
        else:
            checkpoint_path = os.path.join(checkpoint_base_path, f'cnn_model_epoch_{epoch}.pt')
            
        if not os.path.exists(checkpoint_path):
            print(f"Skipping epoch {epoch} - checkpoint not found at {checkpoint_path}")
            continue
            
        print(f"Processing epoch {epoch}")
        # Load checkpoint without weights_only parameter
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        
        # Evaluate on train and test sets
        train_metrics = evaluate_checkpoint(model, train_loader, device)
        test_metrics = evaluate_checkpoint(model, test_loader, device)
        
        # Store metrics
        metrics['epochs'].append(epoch)
        metrics['train_accuracies'].append(train_metrics['accuracy'])
        metrics['test_accuracies'].append(test_metrics['accuracy'])
        metrics['train_losses'].append(train_metrics['loss'])
        metrics['test_losses'].append(test_metrics['loss'])
        metrics['train_confidences'].append(train_metrics['avg_confidence'])
        metrics['test_confidences'].append(test_metrics['avg_confidence'])
        metrics['train_confidence_distributions'].append(train_metrics['confidences'])  # Already a list
        metrics['test_confidence_distributions'].append(test_metrics['confidences'])    # Already a list
    
    # Save metrics to JSON
    with open(json_save_path, 'w') as f:
        json_metrics = {
            'train_accuracies': [float(x) for x in metrics['train_accuracies']],
            'test_accuracies': [float(x) for x in metrics['test_accuracies']],
            'train_losses': [float(x) for x in metrics['train_losses']],
            'test_losses': [float(x) for x in metrics['test_losses']],
            'train_confidences': [float(x) for x in metrics['train_confidences']],
            'test_confidences': [float(x) for x in metrics['test_confidences']],
            'epochs': metrics['epochs'],
            'train_confidence_distributions_stats': [
                {
                    'mean': float(np.mean(dist)),
                    'std': float(np.std(dist)),
                    'min': float(np.min(dist)),
                    'max': float(np.max(dist)),
                    'median': float(np.median(dist)),
                    'q1': float(np.percentile(dist, 25)),
                    'q3': float(np.percentile(dist, 75))
                } for dist in metrics['train_confidence_distributions']
            ],
            'test_confidence_distributions_stats': [
                {
                    'mean': float(np.mean(dist)),
                    'std': float(np.std(dist)),
                    'min': float(np.min(dist)),
                    'max': float(np.max(dist)),
                    'median': float(np.median(dist)),
                    'q1': float(np.percentile(dist, 25)),
                    'q3': float(np.percentile(dist, 75))
                } for dist in metrics['test_confidence_distributions']
            ]
        }
        json.dump(json_metrics, f, indent=4)
    
    return metrics

In [9]:
class SmallerComparableCNN(nn.Module):
    def __init__(self):
        super(SmallerComparableCNN, self).__init__()
        # Reduced initial channels and total layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # Reduced from 64 to 32
        self.bn1 = nn.BatchNorm2d(32)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # Reduced from 128 to 64
        self.bn2 = nn.BatchNorm2d(64)
        
        # Removed two conv layers to reduce parameters
        
        # Global average pooling and final dense layer
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, 10)  # Changed input features to match last conv layer
        
    def forward(self, x):
        # First block
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)
        
        # Second block
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)
        
        # Global average pooling
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        
        # Final classification
        logits = self.fc(x)
        probabilities = F.softmax(logits, dim=-1)
        
        return logits, probabilities
    

In [11]:
# Initialize only CNN model for now
cnn_model = SmallerComparableCNN().to(device)

# First reconstruct metrics for CNN only
cnn_metrics = reconstruct_metrics_from_checkpoints(
    model=cnn_model,
    checkpoint_base_path='smaller_cnn_checkpoints',
    train_loader=train_loader,
    test_loader=test_loader,
    json_save_path='New_CNN_Metrics.json',
    is_mamba=False,
    device=device
)

Processing epoch 100


C:\Users\baris\AppData\Local\Temp\ipykernel_9424\1448401691.py:66: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device

Processing epoch 200
Processing epoch 300
Processing epoch 400
Processing epoch 500
Processing epoch 600
Processing epoch 700
Processing epoch 800
Processing epoch 900
Processing epoch 1000
Processing epoch 1100
Processing epoch 1200
Processing epoch 1300
Processing epoch 1400
Processing epoch 1500


In [10]:
# Initialize MAMBA with matching args
args = ModelArgs(
   d_model=64,      
   n_layer=4,        
   vocab_size=0      
)

mamba_model = ImageMamba(args, num_classes=10).to(device)

mamba_metrics = reconstruct_metrics_from_checkpoints(
   model=mamba_model,
   checkpoint_base_path='model_checkpoints_ext_old',
   train_loader=train_loader,
   test_loader=test_loader,
   json_save_path='New_MAMBA_Metrics.json',
   is_mamba=True,
   device=device
)

Processing epoch 100


C:\Users\baris\AppData\Local\Temp\ipykernel_9424\1448401691.py:66: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device

Processing epoch 200
Processing epoch 300
Processing epoch 400
Processing epoch 500
Processing epoch 600
Processing epoch 700
Processing epoch 800
Processing epoch 900
Processing epoch 1000
Processing epoch 1100
Processing epoch 1200
Processing epoch 1300
Processing epoch 1400
Processing epoch 1500


NameError: name 'cnn_metrics' is not defined

In [16]:
plot_metrics_from_json('New_CNN_Metrics.json', 'New_MAMBA_Metrics.json')


Saved comparison plots to comparison_plots\model_comparisons.png
